# Help Desk - Easy

## Scenario
A software company has been successful in selling its products to a number of customer organisations, and there is now a high demand for technical support. There is already a system in place for logging support calls taken over the telephone and assigning them to engineers, but it is based on a series of spreadsheets. With the growing volume of data, using the spreadsheet system is becoming slow, and there is a significant risk that errors will be made.

![rel](https://sqlzoo.net/w/images/3/38/Helpdesk.png)

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1.
There are three issues that include the words "index" and "Oracle". Find the call_date for each of them

```
+---------------------+----------+
| call_date           | call_ref |
+---------------------+----------+
| 2017-08-12 16:00:00 |     1308 |
| 2017-08-16 14:54:00 |     1697 |
| 2017-08-16 19:12:00 |     1731 |
+---------------------+----------+
```

In [2]:
shift <- dbReadTable(con, 'Shift')
staff <- dbReadTable(con, 'Staff')
issue <- dbReadTable(con, 'Issue')
shift_type <- dbReadTable(con, 'Shift_type')
level <- dbReadTable(con, 'Level')
customer <- dbReadTable(con, 'Customer')
caller <- dbReadTable(con, 'Caller')

In [3]:
issue %>%
    filter(str_detect(Detail, 'index') & 
           str_detect(Detail, 'Oracle')) %>%
    select(Call_date, Call_ref)

Call_date,Call_ref
<dttm>,<int>
2017-08-12 16:00:00,1308
2017-08-16 14:54:00,1697
2017-08-16 19:12:00,1731


## 2.
Samantha Hall made three calls on 2017-08-14. Show the date and time for each

```
+---------------------+------------+-----------+
| call_date           | first_name | last_name |
+---------------------+------------+-----------+
| 2017-08-14 10:10:00 | Samantha   | Hall      |
| 2017-08-14 10:49:00 | Samantha   | Hall      |
| 2017-08-14 18:18:00 | Samantha   | Hall      |
+---------------------+------------+-----------+
```

In [4]:
issue %>% 
    inner_join(caller, by=c(Caller_id="Caller_id")) %>%
    filter(First_name=='Samantha' &
           Last_name=='Hall' &
           as.Date(Call_date)==as.Date('2017-08-14')) %>%
    select(Call_date, First_name, Last_name)

Call_date,First_name,Last_name
<dttm>,<chr>,<chr>
2017-08-14 10:10:00,Samantha,Hall
2017-08-14 10:49:00,Samantha,Hall
2017-08-14 18:18:00,Samantha,Hall


## 3.
There are 500 calls in the system (roughly). Write a query that shows the number that have each status.

```
+--------+--------+
| status | Volume |
+--------+--------+
| Closed |    486 |
| Open   |     10 |
+--------+--------+
```

In [5]:
issue %>%
    group_by(Status) %>%
    summarise(volume=n())

Status,volume
<chr>,<int>
Closed,486
Open,10


## 4.
Calls are not normally assigned to a manager but it does happen. How many calls have been assigned to staff who are at Manager Level?

```
+------+
| mlcc |
+------+
|   51 |
+------+
```

In [6]:
issue %>% 
    inner_join(staff, by=c(Assigned_to="Staff_code")) %>%
    inner_join(level, by=c(Level_code="Level_code")) %>%
    filter(Manager=='Y') %>%
    tally %>%
    rename(mlcc='n')

mlcc
<int>
51


## 5.
Show the manager for each shift. Your output should include the shift date and type; also the first and last name of the manager.

```
+------------+------------+------------+-----------+
| Shift_date | Shift_type | first_name | last_name |
+------------+------------+------------+-----------+
| 2017-08-12 | Early      | Logan      | Butler    |
| 2017-08-12 | Late       | Ava        | Ellis     |
| 2017-08-13 | Early      | Ava        | Ellis     |
| 2017-08-13 | Late       | Ava        | Ellis     |
| 2017-08-14 | Early      | Logan      | Butler    |
| 2017-08-14 | Late       | Logan      | Butler    |
| 2017-08-15 | Early      | Logan      | Butler    |
| 2017-08-15 | Late       | Logan      | Butler    |
| 2017-08-16 | Early      | Logan      | Butler    |
| 2017-08-16 | Late       | Logan      | Butler    |
+------------+------------+------------+-----------+
```

In [7]:
shift %>% 
    inner_join(staff, by=c(Manager="Staff_code")) %>%
    mutate(Shift_date=as.Date(Shift_date)) %>%
    distinct(Shift_date, Shift_type, First_name, Last_name) %>%
    arrange(Shift_date, Shift_type)

Shift_date,Shift_type,First_name,Last_name
<date>,<chr>,<chr>,<chr>
2017-08-12,Early,Logan,Butler
2017-08-12,Late,Ava,Ellis
2017-08-13,Early,Ava,Ellis
2017-08-13,Late,Ava,Ellis
2017-08-14,Early,Logan,Butler
2017-08-14,Late,Logan,Butler
2017-08-15,Early,Logan,Butler
2017-08-15,Late,Logan,Butler
2017-08-16,Early,Logan,Butler


In [8]:
dbDisconnect(con)